<a href="https://colab.research.google.com/github/arqMedes/aulas/blob/master/Grafo_Circuitos_Eletricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teoria de Circuitos Elétricos

![](https://github.com/arqMedes/images/blob/main/grafo_circuito_eletrico.jpg?raw=true)


🔹 Etapa 1: Representação de um ramo

Cada ramo precisa guardar informações como:

**tipo:** (Resistor, Capacitor, Indutor, Fonte de Tensão, Fonte de Corrente, etc.)

**valor:** (resistência, capacitância, etc.)

**Vs:** (fonte de tensão equivalente do ramo)

**Is:** (fonte de corrente equivalente do ramo)

**corrente do ramo (I):**

**tensão do ramo (V):**


🔹 Etapa 2: Montagem da lista de ramos

🔹 Etapa 3: Determinar a lista de Nós e a Matriz de Incidência A

Se temos N nós e M ramos, a matriz de incidência A é uma matriz N × M onde:

+1 → ramo sai do nó

-1 → ramo entra no nó

0 → ramo não conectado


🔹 Etapa 4: Montar Vs, Is, G

🔹 Etapa 5: Cálculos (Método dos Nós Modificado)

$$G_b=A⋅G⋅A.T \\
𝐼_𝑛 = 𝐴⋅(𝐺⋅𝑉_𝑠−𝐼_𝑠) \\
E=G_b^{−1}.In \\
V=A.T⋅E  \\
I=I_s+G⋅(V−V_s)
$$

Implementação:


🔹 Exemplo de circuito

Fonte de tensão de 10 V (nó 1 para nó 0, que é a referência).

Dois resistores em série ligados a essa fonte:

R1 = 10 Ω (entre nó 1 e nó 2)

R2 = 5 Ω (entre nó 2 e nó 0)

Isso forma um divisor de tensão bem clássico.


🔹 Resultado esperado

Como é um divisor de tensão de 10 V com resistores 10 Ω e 5 Ω:



In [ ]:
class Ramo:
    def __init__(self, tipo, valor, Vs=0.0, Is=0.0):
        self.tipo = tipo
        self.valor = valor
        self.Vs = Vs   # fonte de tensão no ramo
        self.Is = Is   # fonte de corrente no ramo
        self.I = 0.0   # corrente calculada
        self.V = 0.0   # tensão calculada

    def condutancia(self):
        if self.tipo.lower() == "resistor" and self.valor != 0:
            return 1.0 / self.valor
        return 0.0


In [ ]:
circuito = [
    Ramo("Resistor", 10),
    Ramo("Resistor", 5),
    Ramo("Fonte de Tensão", 0, Vs=10),
    Ramo("Fonte de Corrente", 0, Is=2)
]


In [ ]:
import numpy as np

# Exemplo: 3 nós (0 é referência), 4 ramos
A = np.array([
    [ 1, -1,  0,  0],  # nó 1
    [-1,  0,  1, -1],  # nó 2
    [ 0,  1, -1,  1]   # nó 3
], dtype=float)


In [ ]:
Vs = np.array([r.Vs for r in circuito], dtype=float)  # fontes de tensão
Is = np.array([r.Is for r in circuito], dtype=float)  # fontes de corrente
G  = np.diag([r.condutancia() for r in circuito])     # condutância diagonal


In [ ]:
# 1) Gb
Gb = A @ G @ A.T

# 2) In
In = A @ (G @ Vs - Is)

# 3) E (tensões nos nós)
E = np.linalg.solve(Gb, In)

# 4) V (tensões nos ramos)
V = A.T @ E

# 5) I (correntes nos ramos)
I = Is + G @ (V - Vs)

# Atualiza ramos com resultados
for k, r in enumerate(circuito):
    r.V = V[k]
    r.I = I[k]


In [ ]:
print("Tensões nos nós (E):", E)
print("Tensões nos ramos (V):", V)
print("Correntes nos ramos (I):", I)

for idx, r in enumerate(circuito):
    print(f"Ramo {idx+1} -> Tipo: {r.tipo}, V={r.V:.3f}, I={r.I:.3f}")


Tensões nos nós (E): [ -6.  14. -16.]
Tensões nos ramos (V): [-20. -10.  30. -30.]
Correntes nos ramos (I): [-2. -2.  0.  2.]
Ramo 1 -> Tipo: Resistor, V=-20.000, I=-2.000
Ramo 2 -> Tipo: Resistor, V=-10.000, I=-2.000
Ramo 3 -> Tipo: Fonte de Tensão, V=30.000, I=0.000
Ramo 4 -> Tipo: Fonte de Corrente, V=-30.000, I=2.000
